In [1]:
import sys
mmseg_path = "../../submodules/mmsegmentation/"
sys.path.append(mmseg_path)

#check mmseg installation
import os
import torch, torchvision
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
import mmseg
from mmseg.utils import register_all_modules
from mmseg.apis import inference_model, init_model

from mmengine import Config
from mmengine.runner import Runner

#check versions of mmseg, mmcv, and mmsegmentation
print('Pytorch Version', torch.__version__)
print('CUDA available',torch.cuda.is_available())

print('MMCV Version', mmcv.__version__)
print('CUDA Version', get_compiling_cuda_version())
print('Compiler Version', get_compiler_version())

print('mmsegmentation Version', mmseg.__version__)

Pytorch Version 1.13.1+cu117
CUDA available True
MMCV Version 2.0.1
CUDA Version 11.7
Compiler Version GCC 9.4
mmsegmentation Version 1.2.1


In [2]:
#create a dictionary of configurations and their respective checkpoints
configs = [
    {
        "config_name":"unet-s5-d16_deeplabv3_4xb4-40k_DME-128x128.py",
        "checkpoint":"deeplabv3_unet_s5-d16_128x128_40k_stare_20201226_094047-93dcb93c.pth"
    },
    {
        "config_name":"unet-s5-d16_deeplabv3_4xb4-ce-1.0-dice-3.0-40k_DME-128x128.py",
        "checkpoint":"deeplabv3_unet_s5-d16_ce-1.0-dice-3.0_128x128_40k_stare_20211210_201825-21db614c.pth"
    },
    {
        "config_name": "unet-s5-d16_fcn_4xb4-40k_DME-128x128.py",
        "checkpoint": "fcn_unet_s5-d16_128x128_40k_stare_20201223_191051-7d77e78b.pth"
    },
    {
        "config_name": "unet-s5-d16_fcn_4xb4-ce-1.0-dice-3.0-40k_DME-128x128.py",
        "checkpoint":"fcn_unet_s5-d16_ce-1.0-dice-3.0_128x128_40k_stare_20211210_201821-f75705a9.pth"
    },
    {
        "config_name":"unet-s5-d16_pspnet_4xb4-40k_DME-128x128.py",
        "checkpoint":"pspnet_unet_s5-d16_128x128_40k_stare_20201227_181818-3c2923c4.pth"
    },
    {
        "config_name":"unet-s5-d16_pspnet_4xb4-ce-1.0-dice-3.0-40k_DME-128x128.py",
        "checkpoint":"pspnet_unet_s5-d16_ce-1.0-dice-3.0_128x128_40k_stare_20211210_201823-f1063ef7.pth"
    }
]

In [4]:
for num in range(6):
    print(num,configs[num]["config_name"])
    #initialize the configuration file
    config_idx = num
    config_folder = "ece661_devp"
    config_name = configs[config_idx]["config_name"]
    cfg_path = os.path.join(mmseg_path,"configs",config_folder,config_name)
    cfg = Config.fromfile(cfg_path)

    #set the working directory
    cfg.work_dir = os.path.join(mmseg_path,'work_dirs',config_name)

    #set the checkpoint to load from
    checkpoint_name = configs[config_idx]["checkpoint"]
    checkpoint_path = os.path.join(mmseg_path,'checkpoints',checkpoint_name)
    cfg.load_from = checkpoint_path

    #specify the evaluation metrics to use
    cfg.val_evaluator.iou_metrics= ['mIoU', 'mDice', 'mFscore']
    cfg.test_evaluator.iou_metrics = ['mIoU', 'mDice', 'mFscore']

    #initialize the runner
    runner = Runner.from_cfg(cfg)

    #perform the testing
    runner.test()


0 unet-s5-d16_deeplabv3_4xb4-40k_DME-128x128.py
12/06 12:46:38 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 2045138786
    GPU 0: NVIDIA RTX A5000
    CUDA_HOME: /usr/local/cuda
    NVCC: Not Available
    GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 1.13.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute

/bin/sh: 1: /usr/local/cuda/bin/nvcc: not found


12/06 12:46:38 - mmengine - INFO - Config:
norm_cfg = dict(type='SyncBN', requires_grad=True)
data_preprocessor = dict(
    type='SegDataPreProcessor',
    mean=[123.675, 116.28, 103.53],
    std=[58.395, 57.12, 57.375],
    bgr_to_rgb=True,
    pad_val=0,
    seg_pad_val=255,
    size=(128, 128))
model = dict(
    type='EncoderDecoder',
    data_preprocessor=dict(
        type='SegDataPreProcessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True,
        pad_val=0,
        seg_pad_val=255,
        size=(128, 128)),
    pretrained=None,
    backbone=dict(
        type='UNet',
        in_channels=3,
        base_channels=64,
        num_stages=5,
        strides=(1, 1, 1, 1, 1),
        enc_num_convs=(2, 2, 2, 2, 2),
        dec_num_convs=(2, 2, 2, 2),
        downsamples=(True, True, True, True),
        enc_dilations=(1, 1, 1, 1, 1),
        dec_dilations=(1, 1, 1, 1),
        with_cp=False,
        conv_cfg=None,
        norm_cfg=

/home/djp69/miniconda3/envs/ECE661_mmseg/lib/python3.10/site-packages/mmengine/utils/manager.py:113: UserWarning: <class 'mmseg.visualization.local_visualizer.SegLocalVisualizer'> instance named of visualizer has been created, the method `get_instance` should not accept any other arguments
  warnings.warn(


12/06 12:46:39 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
12/06 12:46:39 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

...

12/06 12:48:40 - mmengine - INFO - Config:
norm_cfg = dict(type='SyncBN', requires_grad=True)
data_preprocessor = dict(
    type='SegDataPreProcessor',
    mean=[123.675, 116.28, 103.53],
    std=[58.395, 57.12, 57.375],
    bgr_to_rgb=True,
    pad_val=0,
    seg_pad_val=255,
    size=(128, 128))
model = dict(
    type='EncoderDecoder',
    data_preprocessor=dict(
        type='SegDataPreProcessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True,
        pad_val=0,
        seg_pad_val=255,
        size=(128, 128)),
    pretrained=None,
    backbone=dict(
        type='UNet',
        in_channels=3,
        base_channels=64,
        num_stages=5,
        strides=(1, 1, 1, 1, 1),
        enc_num_convs=(2, 2, 2, 2, 2),
        dec_num_convs=(2, 2, 2, 2),
        downsamples=(True, True, True, True),
        enc_dilations=(1, 1, 1, 1, 1),
        dec_dilations=(1, 1, 1, 1),
        with_cp=False,
        conv_cfg=None,
        norm_cfg=

/home/djp69/miniconda3/envs/ECE661_mmseg/lib/python3.10/site-packages/mmengine/utils/manager.py:113: UserWarning: <class 'mmseg.visualization.local_visualizer.SegLocalVisualizer'> instance named of visualizer has been created, the method `get_instance` should not accept any other arguments
  warnings.warn(


12/06 12:48:41 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
12/06 12:48:41 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

/home/djp69/ECE661_mmSeg/notebooks/dev/../../submodules/mmsegmentation/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/home/djp69/ECE661_mmSeg/notebooks/dev/../../submodules/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/home/djp69/ECE661_mmSeg/notebooks/dev/../../submodules/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:249: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


12/06 12:49:02 - mmengine - WARNING - The prefix is not set in metric class IoUMetric.
Loads checkpoint by local backend from path: ../../submodules/mmsegmentation/checkpoints/pspnet_unet_s5-d16_ce-1.0-dice-3.0_128x128_40k_stare_20211210_201823-f1063ef7.pth
12/06 12:49:02 - mmengine - INFO - Load checkpoint from ../../submodules/mmsegmentation/checkpoints/pspnet_unet_s5-d16_ce-1.0-dice-3.0_128x128_40k_stare_20211210_201823-f1063ef7.pth
12/06 12:49:04 - mmengine - INFO - per class results:
12/06 12:49:04 - mmengine - INFO - 
+------------+-------+-------+-------+--------+-----------+--------+
|   Class    |  IoU  |  Acc  |  Dice | Fscore | Precision | Recall |
+------------+-------+-------+-------+--------+-----------+--------+
| background | 98.75 | 99.95 | 99.37 | 99.37  |    98.8   | 99.95  |
|   vessel   |  0.0  |  nan  |  0.0  |  nan   |    0.0    |  nan   |
+------------+-------+-------+-------+--------+-----------+--------+
12/06 12:49:04 - mmengine - INFO - Iter(test) [8/8]    a